# Options Data Analysis and Visualization 

## Importing Modules

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly as pt
import plotly.express as px

## Data Cleaning and Transformation

In [2]:
# Import the web oi data
web_oi_data = pd.read_csv('Data/BN_2024Jan15_Expiry_2024Jan31.csv')
web_oi_data


,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID,ASK,ASK QTY,...,BID QTY.1,BID.1,ASK.1,ASK QTY.1,CHNG.1,LTP.1,IV.1,VOLUME.1,CHNG IN OI.1,OI.1
0,-,-,-,-,-,-,30.0,8709.95,8965.45,750.00,...,900.00,0.15,-,-,-,-,-,-,-,-
1,-,-,-,-,-,-,750.0,8166.45,8388.15,30.00,...,900.00,0.15,-,-,-,-,-,-,-,-
2,-,-,-,-,-,-,30.0,7719.55,7899.25,30.00,...,900.00,0.15,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,750.0,7176.55,7347.75,45.00,...,900.00,1.05,-,-,-,-,-,-,-,-
4,293.00,-,-,-,-,-,45.0,6731.40,6894.50,45.00,...,900.00,1.05,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,202.00,-38.00,71.00,21.94,10.85,0.55,210.0,10.90,13.55,75.00,...,750.00,5077.75,5219.50,60.00,-,-,-,-,-,5.00
86,647.00,-25.00,211.00,23.60,10.70,1.30,75.0,10.25,10.70,480.00,...,750.00,5533.60,5752.70,60.00,-,-,-,-,-,-
87,-,-,-,-,-,-,5400.0,3.05,-,-,...,45.00,6082.65,6251.80,45.00,-,-,-,-,-,-
88,-,-,-,-,-,-,2700.0,3.05,-,-,...,750.00,6574.70,6755.90,45.00,-,-,-,-,-,-


In [3]:
def call_modification(trimmed_oi_data):
    
    call_options = trimmed_oi_data[['STRIKE','BID QTY','BID','ASK','ASK QTY','CHNG','LTP','IV','VOLUME','CHNG IN OI','OI']]
    call_options["STRIKE"] = call_options["STRIKE"].astype(int).astype(str)
    call_options['CE/PE'] = ['CE' for i in call_options['STRIKE']]
    call_options['STRIKE'] = call_options['STRIKE'] + call_options['CE/PE']

    return call_options
    
def put_modification(trimmed_oi_data):
    
    put_options = trimmed_oi_data[['STRIKE', 'BID QTY.1', 'BID.1', 'ASK.1', 'ASK QTY.1','CHNG.1', 'LTP.1', 'IV.1', 'VOLUME.1', 'CHNG IN OI.1', 'OI.1']]
    put_options.columns = ['STRIKE','BID QTY','BID','ASK','ASK QTY','CHNG','LTP','IV','VOLUME','CHNG IN OI','OI']
    put_options["STRIKE"] = put_options["STRIKE"].astype(int).astype(str)
    put_options['CE/PE'] = ['PE' for i in put_options['STRIKE']]
    put_options['STRIKE'] = put_options['STRIKE'] + put_options['CE/PE']

    return put_options


def oi_retriever(raw_oi_dataframe,symbol):
    "Converts Raw Web OI data to useful dataframe for analysis"
    
    # Getting the latest close
    latest = yf.download(symbol)
    last_close = latest['Close'].iloc[-1].round()
    last_close = round(last_close/100)*100
     
    current_index = np.where(raw_oi_dataframe == last_close)[0][0]
    trimmed_oi_data = raw_oi_dataframe.iloc[current_index-10:current_index+11]

    call_oi = call_modification(trimmed_oi_data)
    put_oi = put_modification(trimmed_oi_data)
    
    final_dataset = call_oi.append(put_oi,ignore_index=True)
    return final_dataset


In [ ]:
# Retrieve the necessary option contracts from raw data
oc_for_analysis = oi_retriever(web_oi_data,'^NSEBANK')

# Exporting The CSV
oc_for_analysis.to_csv(r"Data/BN_2024Jan15_Expiry_2024Jan31_processed.csv", index=False)

## Data Visualization

In [5]:
option_contracts = pd.read_csv('Data/BN_2024Jan15_Expiry_2024Jan31_processed.csv')

# Overall CE/PE
overall_info = pd.DataFrame()
overall_info['Volume'] = (option_contracts.groupby(['CE/PE'])['VOLUME'].sum().values)
overall_info['CE/PE'] = ['CE','PE']
overall_info

# Vizualisation of Overall CE/PE
fig = px.pie(overall_info, values='Volume', names='CE/PE', color='CE/PE' ,color_discrete_map={'CE':'#79EA86','PE':'#e75757'},title='Overall Market Direction')
fig.show()

# CE PERFORMANCE
ce_data = option_contracts[option_contracts['CE/PE'] == 'CE']

# CE BID vs. ASK
ce_info = pd.DataFrame()
ce_info['Action'] = ['BID','ASK']
ce_info['Quantity'] = [ce_data['BID QTY'].sum(),ce_data['ASK QTY'].sum()]

# Visualization
fig = px.pie(ce_info, values='Quantity', names='Action', color='Action' ,color_discrete_map={'BID':'#79EA86','ASK':'#e75757'},title='CE Bid vs. ASK')
fig.show()

# STRIKE vs. CHNG IN OI
fig = px.bar(ce_data,x='STRIKE',y='CHNG IN OI')
fig.show()

# STRIKE vs. CHNG IN PRICE
price_info = pd.DataFrame()
price_info['STRIKE'] = ce_data['STRIKE']
price_info['PCHNG'] = (ce_data['CHNG']/(ce_data['LTP']-ce_data['CHNG']))*100

fig = px.bar(price_info,x='STRIKE',y='PCHNG')
fig.show()

# CE INDIVIDUAL BID vs. ASK
df = ce_data[['STRIKE','BID QTY','ASK QTY']]
df = df.reset_index()
df = pd.melt(df, id_vars='STRIKE', value_vars=['BID QTY','ASK QTY'])

# Bar Mode
fig = px.bar(df,x='STRIKE',y='value',color='variable',color_discrete_map={'BID QTY':'#79EA86','ASK QTY': '#e75757'},barmode='group',title='STRIKE: BID vs. ASK')
fig.show()

# PE PERFORMANCE
pe_data = option_contracts[option_contracts['CE/PE'] == 'PE']

# PE BID vs. ASK
pe_info = pd.DataFrame()
pe_info['Action'] = ['BID','ASK']
pe_info['Quantity'] = [pe_data['BID QTY'].sum(),pe_data['ASK QTY'].sum()]

# Visualization
fig = px.pie(pe_info, values='Quantity', names='Action', color='Action' ,color_discrete_map={'BID':'#79EA86','ASK':'#e75757'},title='PE Bid vs. ASK')
fig.show()

# STRIKE vs. CHNG IN OI
fig = px.bar(pe_data,x='STRIKE',y='CHNG IN OI')
fig.show()

# STRIKE vs. CHNG IN PRICE
price_info = pd.DataFrame()
price_info['STRIKE'] = pe_data['STRIKE']
price_info['PCHNG'] = (pe_data['CHNG']/(pe_data['LTP']-pe_data['CHNG']))*100

fig = px.bar(price_info,x='STRIKE',y='PCHNG')
fig.show()

# PE INDIVIDUAL BID vs. ASK
df = pe_data[['STRIKE','BID QTY','ASK QTY']]
df = df.reset_index()
df = pd.melt(df, id_vars='STRIKE', value_vars=['BID QTY','ASK QTY'])

# Bar Mode
fig = px.bar(df,x='STRIKE',y='value',color='variable',color_discrete_map={'BID QTY':'#79EA86','ASK QTY': '#e75757'},barmode='group',title='STRIKE: BID vs. ASK')
fig.show()